In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import importlib.util

In [2]:
device = torch.device("cuda:3") # "cuda:3" if torch.cuda.is_available() else 


In [3]:
device

device(type='cuda', index=3)

In [14]:
# Model Architecture

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            # nn.Sigmoid(),
            # nn.Linear(256, 128),
            # nn.ReLU(),
            # nn.Linear(64, 32),
            # nn.ReLU(),
            # nn.Linear(64, 32),  # Output layer with 1 unit for regression
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)




In [8]:
# training dataset

LATENT_SIZE = 512
MAX_VARIANCE = 5

# specify the module that needs to be
# imported relative to the path of the
# module
latent_load_module = importlib.util.spec_from_file_location("load_latent","/home/sjay9734/diff_encoder/face_privacy/face-privacy-diffae/utils/load_latent.py")
 
# creates a new module based on spec
latent_load = importlib.util.module_from_spec(latent_load_module)
 
# executes the module in its own namespace
# when a module is imported or reloaded.
latent_load_module.loader.exec_module(latent_load)

X_train, y_train, X_test, y_test = latent_load.pre_process_celebA(TRAINING_AMOUNT = 0.9)

def create_dataset(axis = 0):
    i = 0
    train_X_list = []
    train_y_list = []

    while i < (20000):
        train_y_list.append(X_train[i][axis])
        if np.random.randint(2, size=1) == 1:
            train_X_list.append(X_train[i][:100])
            i += 1
        else:
            random_index = np.random.randint(len(X_train), size=1)
            gaussian_noise = np.random.normal(loc=0.0, 
                                            scale=np.random.randint(MAX_VARIANCE, size=1), size=LATENT_SIZE)
            train_X_list.append(np.reshape(X_train[random_index] + gaussian_noise, (LATENT_SIZE))[:100])
            
    augmented_train_X = np.array(train_X_list)
    augmented_train_y = np.array(train_y_list)

    return augmented_train_X, augmented_train_y

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
model_list = []

for i in range(10):
    input_data, target_data = create_dataset(i)
    
    model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=10000)

    # Train the model on the training data
    model.fit(input_data, target_data)

    # Make predictions on the test data
    # y_pred = model.predict(X_test)
    model_list.append(model)

In [10]:
for i in range(10):
    for axis in range(5):
        prediction = model_list[axis].predict(np.reshape(X_test[i][:100], (1,-2)))
        print("data ", i, " axis ", axis, prediction, X_test[i][axis])

data  0  axis  0 [-0.04206394] -0.02080943062901497
data  0  axis  1 [-0.05350521] -0.032778050750494
data  0  axis  2 [0.06646017] 0.07523944228887558
data  0  axis  3 [-0.08459086] -0.08224750310182571
data  0  axis  4 [-0.10204965] -0.12100885063409805
data  1  axis  0 [-0.14547408] -0.15350981056690216
data  1  axis  1 [-0.07620635] -0.07667911052703857
data  1  axis  2 [-0.1005999] -0.10543038696050644
data  1  axis  3 [0.01905925] 0.035152070224285126
data  1  axis  4 [-0.09698107] -0.11162299662828445
data  2  axis  0 [-0.09492698] -0.10565115511417389
data  2  axis  1 [-0.06279798] -0.11371476203203201
data  2  axis  2 [-0.0844908] -0.08033155649900436
data  2  axis  3 [-0.06902674] -0.0699574425816536
data  2  axis  4 [-0.04286962] -0.04742433875799179
data  3  axis  0 [-0.08197297] -0.08760935068130493
data  3  axis  1 [-0.05047941] -0.010711896233260632
data  3  axis  2 [-0.10577694] -0.11240431666374207
data  3  axis  3 [-0.05541274] -0.06713921576738358
data  3  axis  4 [-

In [6]:
batch_size = 32
num_epochs = 300

model_list = []

for i in range(10):
    input_data, target_data = create_dataset(i)
    input_data = torch.from_numpy(input_data).type(torch.float32).to(device)
    target_data = torch.from_numpy(target_data).type(torch.float32).to(device)

    # Create a DataLoader to load data in batches
    dataset = TensorDataset(input_data, target_data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = RegressionModel().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        # Forward pass
        for inputs, targets in dataloader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print loss for this epoch
        print(f"axis {i} Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss.item()}")
    
    model_list.append(model)

/home/sjay9734/diff_encoder/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


axis 0 Epoch [1/300] - Loss: 0.005546794272959232
axis 0 Epoch [2/300] - Loss: 0.004931702744215727
axis 0 Epoch [3/300] - Loss: 0.005132718943059444
axis 0 Epoch [4/300] - Loss: 0.0052490332163870335
axis 0 Epoch [5/300] - Loss: 0.005772130563855171
axis 0 Epoch [6/300] - Loss: 0.005499762017279863
axis 0 Epoch [7/300] - Loss: 0.006664359010756016
axis 0 Epoch [8/300] - Loss: 0.0077918460592627525
axis 0 Epoch [9/300] - Loss: 0.005537543445825577
axis 0 Epoch [10/300] - Loss: 0.005398898385465145
axis 0 Epoch [11/300] - Loss: 0.007859368808567524
axis 0 Epoch [12/300] - Loss: 0.0069214096292853355
axis 0 Epoch [13/300] - Loss: 0.005055159330368042
axis 0 Epoch [14/300] - Loss: 0.005782888270914555
axis 0 Epoch [15/300] - Loss: 0.00585480872541666
axis 0 Epoch [16/300] - Loss: 0.009033342823386192
axis 0 Epoch [17/300] - Loss: 0.006570471450686455
axis 0 Epoch [18/300] - Loss: 0.006360987201333046
axis 0 Epoch [19/300] - Loss: 0.007585431449115276
axis 0 Epoch [20/300] - Loss: 0.009509

In [7]:
!nvidia-smi

Thu Sep 21 21:17:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:01:00.0 Off |                  N/A |
| 30%   50C    P2             132W / 320W |   8142MiB / 10240MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
for i in range(10):
    for axis in range(5):
        prediction = model_list[axis](torch.from_numpy(X_test[i][:4]).type(torch.float32).to(device))
        print("data ", i, " axis ", axis, prediction, X_test[i][axis])

NameError: name 'device' is not defined

In [11]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Generate synthetic data (replace this with your actual data)
np.random.seed(0)
X = np.random.rand(100, 32)  # 100 samples with 32 features each
y = 2 * X.sum(axis=1) + np.random.randn(100)  # A simple linear relationship with noise

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an MLPRegressor model
model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=10000)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 7.32
R-squared: 0.39
